In [20]:

!pip install statsmodels
!pip install matplotlib
!pip install pylab

  Could not find a version that satisfies the requirement pylab (from versions: )
No matching distribution found for pylab


In [21]:
!pip install yahoo_fin

In [ ]:
from tkinter import *
class Page(object):
    def __init__(self):
        pass
#build a input page    
    def InputPage(self,root):
        root.title('Finance SuperMan')
        #root.geometry('300*100')
        self.user_input=StringVar()
        self.user_ouput=StringVar()
        
        #self.stoklist=self.read_stocklist
        #CREATE ATTRIBUTES FOR THIS INTERFACE
        self.title_1=Label(root,text="Welcome to Finance SuperMAN!")
        self.entry_label=Label(root,text="Enter a company: ")
        self.entry_box=Entry(root,textvariable=self.user_input)
        self.confirmButton=Button(root,text="confirm",
                                  command=self.OutputPage)
        #LAYOUTS THE WIDGETS
        self.title_1.grid(row=0,sticky=W)
        self.entry_label.grid(row=3,column=0,sticky=W)
        self.entry_box.grid(row=3,column=3,columnspan=2)
        self.confirmButton.grid(row=16,column=4,sticky=E)
    
#build a outputpage        
    def OutputPage(self):
        from tkinter import messagebox
        output=Toplevel()
        output.title('in-time Information')
        output.geometry('300x1000')
        title_1=Label(output,text="Welcome to Finance SuperMAN!")
        title_1.grid(row=0,sticky=W)
        ref=self.read_stocklist()
        ipt=self.user_input.get().upper()
        if ipt not in ref:
            messagebox.showinfo('enter error','please enter the right ticket name')
        else:
            #option button
            Button_company_information=Button(output,text="Company Basic Informations",
                                      command=lambda:self.get_summary(ipt))
            Button_competitor=Button(output,text="competitor analysis",
                                      command=lambda:self.comp_stat_table(ipt))
            Button_sector=Button(output,text="sector analysis",
                                      command=lambda:self.get_sector(ipt))
            Button_experts_analysis=Button(output,text="Experts Analyst Report",
                                      command=lambda:self.get_analysis(ipt))
            Button_expected_return=Button(output,text="View history expected returns",
                                      command=lambda:self.expected_returns(ipt))

            Button_Technical_Plot=Button(output,text="Technical Plot",
                                      command=lambda:self.plot_key_technical(ipt))
            Button_correlation=Button(output,text="Stock Correlation",
                                     command=lambda:self.correlation(ipt))
            Button_performance=Button(output,text="Stock overall performance based on S&P500",
                                     command=lambda:self.plot_prices(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
            Button_company_information.grid(row=4,column=0,sticky=W)
            Button_competitor.grid(row=5,column=0,sticky=W)
            Button_sector.grid(row=6,column=0,sticky=W)
            Button_experts_analysis.grid(row=7,column=0,sticky=W)
            Button_expected_return.grid(row=8,column=0,sticky=W)
            Button_Technical_Plot.grid(row=9,column=0,sticky=W)
            Button_correlation.grid(row=10,column=0,sticky=W)
            Button_performance.grid(row=11,column=0,sticky=W)
            

#read in the company list        
    def read_stocklist(self):
        import pandas as pd
        company=pd.DataFrame()
        company=pd.read_csv('companylist.csv',usecols=['Symbol'])
        stocklist=company.append(pd.read_csv('companylist-2.csv',usecols=['Symbol'],header=0))
        stocklist.to_csv('stocklist.csv')
        stocklist=stocklist['Symbol'].tolist()
        return stocklist
#============================ stock basic information ==================================================
    def get_summary(self,ticker):
        import requests
        from bs4 import BeautifulSoup
        url = "https://finance.yahoo.com/quote/"+ ticker + "/profile?p=" + ticker
        response = requests.get(url)
        results_page = BeautifulSoup(response.content,'lxml')
        summary = results_page.find_all('p',class_="Mt(15px) Lh(1.6)")
        if summary == []:
            return "Ticker Error, Please Check if the Ticker is Entered Correctly"
        display(summary[0].get_text())
        self.get_recent_news(ticker)
    
    def get_recent_news(self,ticker):
        import requests
        from bs4 import BeautifulSoup
        url = "https://eresearch.fidelity.com/eresearch/evaluate/snapshot.jhtml?symbols=" + ticker
        response = requests.get(url)
        results_page = BeautifulSoup(response.content,'lxml')
        summary = results_page.find_all('li',class_ = "news-item")
        newslist = list()
        import re
        for i in summary:
            try:
                newstext = re.sub('[^A-Za-z0-9]+', ' ', i.get_text())
                newsurl = "https://eresearch.fidelity.com" + i.find_all('a')[1].get('href')
                newslist.append([newstext[1:re.search(r'(\d{2}) (\d{2}) 2018',newstext).span()[0]],newsurl])
            except:
                pass
        print('Recent News:')
        for i in newslist:
            print(i[0])
            print('Link: ' + i[1])

#============================= stock analyst expectations =============================================
    def get_analysis(self,ticker):
        from IPython.display import display
        from yahoo_fin import stock_info as si
        import pandas as pd
        df = si.get_analysts_info(ticker)
        for i in list(df.keys()):
            display(pd.DataFrame(df.get(i)))

#============================ expected returns table ==================================================
    def get_prices(self,ticker):
        import datetime
        import dateutil.relativedelta
        import numpy as np
        import pandas_datareader.data as web

        stocks = ticker, self.get_competitors(ticker)
        five_years_ago = (datetime.date.today() - dateutil.relativedelta.relativedelta(years=5)).strftime('%Y-%m-%d')
        today = datetime.date.today().strftime('%Y-%m-%d')

        prices = web.DataReader(np.hstack(('SPY',)+stocks),'yahoo',five_years_ago,today)['Adj Close']
        return prices
    def expected_returns(self,ticker):
        import matplotlib.pyplot as plt
        %matplotlib inline
        import pylab as plot

        returns = self.get_prices(ticker).pct_change()

        plt.xlabel('Expected Annual Returns (%)')
        plt.ylabel('Standard Deviations')
        for label,x,y in zip(returns.columns,returns.mean()*365,returns.std()):
            plt.annotate(
                label, 
                xy = (x,y),
                xytext = (10,10),
                textcoords = 'offset points',
                bbox = dict(fc = 'yellow'),
                arrowprops = dict(arrowstyle = '-')
            )
        plot.ylim([min(returns.std()),max(returns.std())])
        plot.xlim([min(returns.mean()*365),max(returns.mean()*365)])
        plt.scatter(returns.mean()*365, returns.std())



#============================= key technical plot ======================================================
    def plot_key_technical(self,ticker):
        %matplotlib inline  

        import pandas_datareader.data as web
        import pandas as pd
        import matplotlib.pyplot as plt
        from matplotlib import dates as mdates
        from mpl_finance import candlestick_ohlc
        import datetime as dt

        symbol = ticker
        data = web.DataReader(symbol, 'yahoo', datetime(2015, 8, 8),datetime.today()) 
        data.reset_index(inplace=True)
        data['Date']=mdates.date2num(data['Date'].astype(dt.date))

        x = data['Date']

        EMA_1_span = 7
        EMA_1 = data['Close'].ewm(span=EMA_1_span,min_periods=EMA_1_span).mean()

        EMA_2_span = 30
        EMA_2 = data['Close'].ewm(span=EMA_2_span,min_periods=EMA_2_span).mean()

        SMA_2_span = EMA_2_span
        SMA_2 = data['Close'].rolling(window=SMA_2_span,center=False).mean()

        MACD = EMA_1 - EMA_2

        # plot

        fig, (ax, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(12,15))

        # plot candlestick, SAM, EMA in subplot_1
        candlestick_ohlc(ax,data.values,width=0.5);
        p1 = ax.plot(x, EMA_1, label='EMA(' + str(EMA_1_span) + ')')
        p2 = ax.plot(x, EMA_2, label='EMA(' + str(EMA_2_span) + ')')
        p3 = ax.plot(x, SMA_2, label='SMA(' + str(SMA_2_span) + ')')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.xaxis.set_major_locator(mdates.MonthLocator([1,4,7,10]))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%b '%y"))
        ax.set_ylabel('Price', fontsize=16)
        ax.legend(loc=4)

        # plot volume in subplot_2
        ax2.bar(x,data['Volume']);
        ax2.set_ylabel('Volume', fontsize=16)

        # plot MACD in subplot_3
        ax3.plot(x, MACD, label='MACD (' + 'EMA(' + str(EMA_1_span) + '), ' + 'EMA(' + str(EMA_2_span) + '))')
        ax3.axhline(0, color='gray', linestyle='--')
        ax3.set_xlabel('Date')
        ax3.set_ylabel('MACD', fontsize=16)
        ax3.legend(loc=4)

#============================= stock correlation========================================================


    def correlation(self,ticker):
        returns = self.get_prices(ticker).pct_change()
        display(returns.corr())

        
#============================= Stock price against S&P 500 with Regression==========================================================       
    
    def plot_prices(self,ticker):
        import statsmodels.api as sm
        import matplotlib.patches as mpatches
        import pylab as plot

        p = self.get_prices(ticker)
        x = p['SPY']
        x = sm.add_constant(x)
        y = p[ticker]
        model = sm.OLS(y,x,missing='drop')
        regression = model.fit()
        print(regression.summary())

        fig,ax = plt.subplots(figsize=(20,10))
        blue_patch = mpatches.Patch(color='blue', label=ticker)
        orange_patch = mpatches.Patch(color='orange', label='S&P 500 (Fitted Value)')
        plt.legend(handles=[blue_patch,orange_patch])
        params = {'legend.fontsize': 15, 'legend.handlelength': 2}
        plot.rcParams.update(params)

        ax.plot(y)
        ax.plot(regression.fittedvalues)

#============================= sector analysis==========================================================        
    def get_sector(self,ticker):
        import requests
        from bs4 import BeautifulSoup
        url = 'https://finance.yahoo.com/quote/' + ticker + '/profile?p=' + ticker

        response = requests.get(url)
        results_page = BeautifulSoup(response.content, 'lxml')
        profile = results_page.find('div', class_='asset-profile-container')
        sector = profile.find_all('strong')[0].get_text()
        industry = profile.find_all('strong')[1].get_text()
        display('Your Sector: '+sector+'    Your Industry: '+industry)
        self.sector_performance()
    
    def sector_data(self,sector_page_link):
        import requests
        from bs4 import BeautifulSoup
        import re

        response = requests.get(sector_page_link)
        results_page = BeautifulSoup(response.content, 'lxml')

        market_cap = results_page.find('div', class_='tab-content-wrapper').find_all('span')[3].get_text()
        market_weight = results_page.find('div', class_='tab-content-wrapper').find_all('span')[5].get_text()

        no_space = r'\S+'
        data = list()
        for i in range(10):
            data.append(re.findall(no_space, results_page.find('div', class_='sec-fundamentals').find_all('td')[i].get_text()))

        pe_last_year = data[0][0]
        pe_this_year = data[1][0]
        ev = data[2][0]
        eps = data[3][0]
        eps_growth = data[4][0]
        revenue_growth = data[5][0]
        roe = data[6][0]
        roi = data[7][0]
        debt_to_equity = data[8][0]
        dividend = data[9][0]
        return market_cap, market_weight, ev, revenue_growth, eps_growth, eps, dividend, pe_last_year, pe_this_year, roe, roi, debt_to_equity

    def sector_performance(self):
        import requests
        from bs4 import BeautifulSoup
        import pandas as pd

        stats_list = list()
        sector_list = list()
        measure_list = list()

        url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"

        results_page = requests.get(url)
        soup = BeautifulSoup(results_page.content, 'lxml')
        all_performance = soup.find_all('a', class_='heading1')

        for sectors in all_performance:
            name = sectors.find('strong').get_text()
            link = "https://eresearch.fidelity.com" + sectors.get('href')
            stats_list.append(self.sector_data(link))
            sector_list.append(name)

        measure_list = ['Market Capital', 'Market Weight', 'Enterprise Value', 'Revenue Growth (TTM)', 'EPS Growth (TTM)', 'EPS (TTM)', 'Dividend Yeild', 'Trailing P/E', 'This Year\'s Estimated P/E', 'ROE (TTM)', 'ROI (TTM)', 'Debt/Equity (TTM)']
        sector_table = pd.DataFrame(stats_list, index=sector_list, columns=measure_list)
        display(sector_table)
        
#========================competitor analysis======================================================================            
    def get_row(self,ticker,measure_list):
        from pandas_datareader import data as web
        a = self.get_financial_stats(ticker)
        row = list()
        row.append(web.DataReader(ticker,"yahoo").iloc[-1,3])
        for i in measure_list:
            if not i == "Price":
                appended = 0
                for j in a:
                    if j[0] == i:
                        row.append(j[1])
                        appended = 1
                if appended == 0:
                    row.append('N/A')   
        return row
    
    def comp_stat_table(self,ticker):
        import pandas as pd
        measure_list = ['Price','Market Cap (intraday)','Enterprise Value','Operating Margin','Quarterly Revenue Growth','Quarterly Earnings Growth','Diluted EPS','Trailing Annual Dividend Yield','5 Year Average Dividend Yield','Payout Ratio','Trailing P/E','Forward P/E','PEG Ratio (5 yr expected)','Return on Equity','Total Debt/Equity','Book Value Per Share','Beta (3Y Monthly)']
        r = self.get_competitors(ticker)
        r.insert(0, ticker)

        s_row = self.get_row(ticker,measure_list)
        a_row = self.get_row(r[1],measure_list)
        b_row = self.get_row(r[2],measure_list)
        c_row = self.get_row(r[3],measure_list)
        d_row = self.get_row(r[4],measure_list)

        comprehensive_list = ['Price', 'Market Cap', 'Enterprise Value', 'Operating Margin', 'Revenue Growth (Quarter)', 'Earnings Growth (Quarter)', 'EPS', 'Dividend Yield', '5-Year Avg Dividend Yield', 'Payout Ratio', 'Trailing P/E', 'Forward P/E', 'PEG', 'ROE (TTM)', 'Debt/Equity', 'Book Value/Share', 'Beta']

        stats_list = [s_row,a_row,b_row,c_row,d_row]
        final_table = pd.DataFrame(stats_list, index=r, columns=comprehensive_list)
    
        display(final_table)

    def get_competitors(self,stock):
        import requests
        from bs4 import BeautifulSoup
        url = 'https://www.nasdaq.com/symbol/' + stock + '/stock-comparison'

        response = requests.get(url)
        results_page = BeautifulSoup(response.content, 'lxml')
        table = results_page.find('div', class_='genTable thin')
        competitors = list()
        for i in range(4):
            competitors.append(table.find_all('input')[i].get('value'))
        return(competitors) 
        
    def get_financial_stats(self,ticker):
        import requests
        from bs4 import BeautifulSoup
        url = "https://finance.yahoo.com/quote/" + ticker + "/key-statistics?p=" + ticker

        response = requests.get(url)
        results_page = BeautifulSoup(response.content,'lxml')
        stats_lists = results_page.find_all('table')
        test_list = list()
        for i in stats_lists:
            stat_name = i.find_all('span')
            stock_stat = i.find_all('td',class_='Fz(s) Fw(500) Ta(end)')
            for j in range(len(stock_stat)):
                if not stat_name[j].get_text() == 'N/A':
                    test_list.append((stat_name[j].get_text(),stock_stat[j].get_text()))
        return test_list
    
    
    
    
root=Tk()
app=Page()
app.InputPage(root)
root.mainloop()

                            OLS Regression Results                            
Dep. Variable:                   AAPL   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                 1.609e+04
Date:                Sun, 02 Dec 2018   Prob (F-statistic):               0.00
Time:                        18:24:55   Log-Likelihood:                -4777.8
No. Observations:                1260   AIC:                             9560.
Df Residuals:                    1258   BIC:                             9570.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -103.4049      1.825    -56.646      0.0

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/barneyxiang/anaconda3/lib/python3.6/tkinter/__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-1-83e569143b8a>", line 55, in <lambda>
    command=lambda:self.plot_prices(ipt))
  File "<ipython-input-1-83e569143b8a>", line 243, in plot_prices
    fig,ax = plt.subplots(figsize=(20,10))
NameError: name 'plt' is not defined
